# Prediction of multistep models with error estimation

To run this notebook you will need to install node.js and the plotly extension for jupyterlab:
```bash
curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -
sudo apt-get install -y nodejs
jupyter labextension install jupyterlab-plotly
pip install jupyterlab-dash
```

In [1]:
import sys
sys.path.insert(0, "/home/iheredia/ignacio/covid/covid-dl/src")
from paths import PATHS
from data.base import make_splits, get_data, single_X_y

In [2]:
import pathlib

import matplotlib
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
import plotly.express as px
import plotly.graph_objects as go

In [3]:
df = get_data(trend=True, multistep=7)
splits = make_splits(df,
                     norm=False,
                     multistep=7
                    )
X, y = single_X_y(splits)

training_end = splits['val']['X'].index[0][0]  # end of training date
validation_end = splits['test']['X'].index[0][0]  # end of validation date

/home/iheredia/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [4]:
# Load predictions
# pred = []
# for f in pathlib.Path('single_step').glob('pred_*'):
#     pred.append(pd.read_csv(f, index_col=['date', 'province']))
# pred = pd.concat(pred, axis=1)
pred = pd.read_csv('multi_step/pred_ff_multistep.csv', header=[0, 1], index_col=[0, 1])

In [5]:
provinces = X.index.get_level_values(1)
dates_dt = X.index.get_level_values(0)
dates_dt = pd.to_datetime(dates_dt).unique()

# Province time series

In [6]:
months_begin = (dates_dt.day == 1)
months_begin = np.arange(len(dates_dt))[months_begin]

marks = {int(i): {'label': dates_dt.strftime('%b %Y')[i]} for i in months_begin}
marks.update({dates_dt.get_loc(training_end): {'label': 'Training ending',
                                               'style': {'color': '#f50'}},
              dates_dt.get_loc(validation_end): {'label': 'Validation ending',
                                                 'style': {'color': '#f50'}}
             })

app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Province trend"),

    dcc.Dropdown(
        id='dropdown',
        clearable=False,
        value=provinces[0],
        options=[{'label': p, 'value': p} for p in provinces.unique()]),
    
    dcc.Slider(id='slider',
               min=0,
               max=len(dates_dt),
               value=dates_dt.get_loc(training_end),
               step=1,
               marks=marks           
),  
    
    dcc.Graph(id='graph'),
])

@app.callback(
    Output('graph', 'figure'),
    [Input('dropdown', 'value'),
     Input('slider', 'value')]
)
def update_figure(province, dateidx):
    
    date = dates_dt[dateidx].strftime('%Y-%m-%d')

    dfx = X.xs(province, level=1, drop_level=True)['incidence 7'].loc[:date]
    dfy = y.xs(province, level=1, drop_level=True).loc[date]
    dfp = pred.xs(province, level=1, drop_level=True).loc[date]

    start = pd.to_datetime(date) + pd.DateOffset(days=-30)
    dend = pd.to_datetime(date) + pd.DateOffset(days=8)
    drange = pd.date_range(date, dend)

    fig = go.Figure(layout={'template': 'seaborn',
                            'hovermode': 'x unified',
                            'xaxis': {'range': [start, dend]}})

    fig.add_trace(go.Scatter(name='historic values',
                             x=dfx.index,
                             y=dfx,
                             line={'color': 'rgba(204, 108, 108, 1)'}
                            )
                 )

    fig.add_trace(go.Scatter(name='upper bound',
                             x=drange,
                             y=dfx[-1:].append(dfp['mean'] + 2 * dfp['std']),
                             mode='lines',
                             line={'width': 0},
                             showlegend=False
                            )
                 )

    fig.add_trace(go.Scatter(name='lower bound',
                             x=drange,
                             y=dfx[-1:].append(dfp['mean'] - 2 * dfp['std']),
                             line={'width': 0},
                             mode='lines',
                             fillcolor='rgba(136, 187, 250, 0.4)',
                             fill='tonexty',
                             showlegend=False
                            )
                 )

    fig.add_trace(go.Scatter(name='predictions',
                             x=drange[1:],
                             y=dfp['mean'],
                             line={'color': 'rgba(70, 128, 199, 1)'}
                            )
                 )

    fig.add_trace(go.Scatter(name='actual values',
                             x=drange,
                             y=[dfx[-1]] + list(dfy),                         
                             line={'color': 'rgba(239, 122, 0, 1)'}
                            )
                 )


    return fig

app.run_server(mode='inline', port=8064)
# app.run_server(mode='external', port=8075)  # bigger